In [1]:
library("stringr")
library("ggtree")
library("ggplot2")
library("ggpmisc")

ggtree v3.10.1 For help: https://yulab-smu.top/treedata-book/

If you use the ggtree package suite in published research, please cite
the appropriate paper(s):

Guangchuang Yu, David Smith, Huachen Zhu, Yi Guan, Tommy Tsan-Yuk Lam.
ggtree: an R package for visualization and annotation of phylogenetic
trees with their covariates and other associated data. Methods in
Ecology and Evolution. 2017, 8(1):28-36. doi:10.1111/2041-210X.12628

Guangchuang Yu.  Data Integration, Manipulation and Visualization of
Phylogenetic Trees (1st edition). Chapman and Hall/CRC. 2022,
doi:10.1201/9781003279242

S Xu, Z Dai, P Guo, X Fu, S Liu, L Zhou, W Tang, T Feng, M Chen, L
Zhan, T Wu, E Hu, Y Jiang, X Bo, G Yu. ggtreeExtra: Compact
visualization of richly annotated phylogenetic data. Molecular Biology
and Evolution. 2021, 38(9):4039-4042. doi: 10.1093/molbev/msab166 

Le chargement a nécessité le package : ggpp

Registered S3 methods overwritten by 'ggpp':
  method                  from   
  heightDetail

In [2]:
trait_model<-readRDS("../corHMM/habitat_fossilized.rds")

df<-read.csv("../../../raw_data/Trait_data_Squaliformes_Fossil.tsv", sep="\t") # omit sep ="\t" for .csv files

phy<-read.tree("../../../raw_data/Squaliformes_fossil.tree")

### Cleaning and preparing data

df$Species<-str_replace(df$Species, " ", "_")

states<-cbind(df$Species, df$Body.size, df$Bioluminescent, df$Habitat)

states_traits<-states[!states[,1] %in% setdiff(states[,1], phy$tip.label),]

states_traits[is.na(states_traits)]<-"?"

states_traits<-states_traits[,c(1, 2, 3)]
colnames(states_traits)<-c("Species", "Continuous", "Categorical")
states_traits<-as.data.frame(states_traits)

In [3]:
node_states_traits<-trait_model$states

### assuming you have no polytomies

node_states<-((length(phy$tip.label)+1):(length(phy$tip.label)+phy$Nnode))

### assuming you have a binary trait    

    if(trait_model$rate.cat == 1){
        col1 <- (node_states_traits[,1])
        col2 <- (node_states_traits[,2])
    }
    if(trait_model$rate.cat == 2){
        col1 <- (node_states_traits[,1]+node_states_traits[,2])
        col2 <- (node_states_traits[,3]+node_states_traits[,4])
    }
    if(trait_model$rate.cat == 3){
        col1 <- (node_states_traits[,1]+node_states_traits[,2]+node_states_traits[,3])
        col2 <- (node_states_traits[,4]+node_states_traits[,5]+node_states_traits[,6])
    }


In [4]:
# Branch
 color_list <- c("#1B9E77", "#D95F02")

# Habitat
# color_list <- c("#5E4FA2", "#74BDCB")

# Bioluminescence
#color_list <- c("#A4BDCA", "#A29C7B")

In [5]:
filename_plot<-"ASE_Habitat_Heterogeneity.pdf"

In [6]:
    node_states_traits<-cbind(col1, col2, node_states)
    colnames(node_states_traits)<-c("Slow", "Fast", "node")
    node_states_traits<-as.data.frame(node_states_traits)
    pies <- nodepie(node_states_traits, cols=1:2, color=color_list, alpha=1)

In [7]:
phy<-ggtree(phy)+
           geom_tiplab(offset = 0.5, size = 2.5, fontface = 4) +
           theme_bw() +
           theme(panel.border = element_blank(),
           legend.key = element_blank(),
           axis.ticks = element_blank(),
           axis.text.y = element_blank(),
           axis.text.x = element_blank(),
           panel.grid = element_blank(),
           panel.grid.minor = element_blank(), 
           panel.grid.major = element_blank(),
           panel.background = element_blank(),
           plot.background = element_rect(fill = "transparent",colour = NA))

In [8]:
    phy_node <- phy %<+% node_states_traits
    phy_complete<-phy_node + geom_plot(data=td_filter(!isTip), mapping=aes(x=x,y=y, label=pies), vp.width=0.03, vp.height=0.03, hjust=0.6, vjust=0.6)
    ASE_plot<-phy_complete + geom_tippoint(data=td_filter(isTip))

In [9]:
ggsave(ASE_plot, filename = filename_plot,  bg = "transparent", width = 10, height = 10)